In [4]:
from skimage import io, color, data
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def kernel_kmeans_segmentation(image, K=5, gamma=0.5, max_iter=100):
    # Load the image
    height, width, channels = image.shape
    
    # Create feature vectors (x, y, R, G, B)
    x_coords = np.arange(width)
    y_coords = np.arange(height)
    xx, yy = np.meshgrid(x_coords, y_coords)
    xx_flat = xx.ravel()
    yy_flat = yy.ravel()
    pixels_flat = image.reshape(-1, channels)
    
    features = np.column_stack([xx_flat, yy_flat, pixels_flat]).astype(np.float64)
    
    # Normalize features
    features[:, 0] /= (width - 1)    # x coordinate
    features[:, 1] /= (height - 1)   # y coordinate
    features[:, 2:] /= 255.0         # RGB values
    
    # Compute RBF kernel matrix
    X = features
    X_norm = np.sum(X ** 2, axis=1)
    K = np.exp(-gamma * (X_norm[:, np.newaxis] + X_norm[np.newaxis, :] - 2 * np.dot(X, X.T)))
    K_diag = np.diag(K)
    
    # Initialize clusters ensuring each cluster has at least one member
    N = X.shape[0]
    clusters = np.zeros(N, dtype=int)
    clusters[:K] = np.arange(K)
    clusters[K:] = np.random.randint(0, K, size=N-K)
    np.random.shuffle(clusters)
    
    for it in range(max_iter):
        print(f"Iteration {it + 1}")
        prev_clusters = clusters.copy()
        
        # Calculate sum1 and sum2 for each cluster
        cluster_ids = np.unique(clusters)
        sum1s = []
        sum2s = []
        sizes = []
        for k in cluster_ids:
            mask = (clusters == k)
            size_k = mask.sum()
            if size_k == 0:
                continue
            sum1 = K[:, mask].sum(axis=1)
            sum2 = K[mask][:, mask].sum()
            sum1s.append(sum1)
            sum2s.append(sum2)
            sizes.append(size_k)
        
        # Handle case where all clusters are empty (unlikely)
        if not sum1s:
            break
        
        # Compute distance matrix
        num_clusters = len(cluster_ids)
        distance_matrix = np.zeros((N, num_clusters))
        for i in range(num_clusters):
            sum1_i = sum1s[i]
            sum2_i = sum2s[i]
            size_i = sizes[i]
            distance_matrix[:, i] = K_diag - 2 * sum1_i / size_i + sum2_i / (size_i ** 2)
        
        # Assign new clusters
        new_clusters_indices = np.argmin(distance_matrix, axis=1)
        new_clusters = cluster_ids[new_clusters_indices]
        
        # Ensure exactly K clusters by reassigning empty clusters
        unique_new = np.unique(new_clusters)
        if len(unique_new) < K:
            present = set(unique_new.tolist())
            for mc in range(K):
                if mc not in present:
                    idx = np.random.choice(N)
                    new_clusters[idx] = mc
        
        clusters = new_clusters
        
        # Check for convergence
        if np.array_equal(clusters, prev_clusters):
            print("Converged!")
            break
    
    # Generate segmented image using cluster average colors
    segmented = np.zeros_like(image)
    flat_image = image.reshape(-1, channels)
    for k in range(K):
        mask = (clusters == k).reshape(height, width)
        if np.any(mask):
            avg_color = np.mean(flat_image[clusters == k], axis=0)
            segmented[mask] = avg_color.astype(int)
    
    return segmented

# Example usage
image = data.astronaut()  # You can also try other images like data.camera(), data.coins(), etc.
image = image[:64,:64,:]

segmented_image = kernel_kmeans_segmentation(image, K=5, gamma=0.5)

# Display results
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(segmented_image)
plt.title('Segmented Image')
plt.axis('off')

plt.show()


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()